In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import TimeDistributedDense, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from keras import backend as K

raw = open('wikigold.conll.txt', 'r').readlines()

all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(' ')
    point.append(stripped_line)
    if line == '\n':
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]

print(point)
print(len(all_x))
print(len(all_x[1]))
print(all_x)

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/pritish/.local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


[]
1840
23
[[['010', 'I-MISC'], ['is', 'O'], ['the', 'O'], ['tenth', 'O'], ['album', 'O'], ['from', 'O'], ['Japanese', 'I-MISC'], ['Punk', 'O'], ['Techno', 'O'], ['band', 'O'], ['The', 'I-ORG'], ['Mad', 'I-ORG'], ['Capsule', 'I-ORG'], ['Markets', 'I-ORG'], ['.', 'O']], [['This', 'O'], ['album', 'O'], ['proved', 'O'], ['to', 'O'], ['be', 'O'], ['more', 'O'], ['commercial', 'O'], ['and', 'O'], ['more', 'O'], ['techno-based', 'O'], ['than', 'O'], ['Osc-Dis', 'I-MISC'], [',', 'O'], ['with', 'O'], ['heavily', 'O'], ['synthesized', 'O'], ['songs', 'O'], ['like', 'O'], ['Introduction', 'I-MISC'], ['010', 'I-MISC'], ['and', 'O'], ['Come', 'I-MISC'], ['.', 'O']], [['Founding', 'O'], ['member', 'O'], ['Kojima', 'I-PER'], ['Minoru', 'I-PER'], ['played', 'O'], ['guitar', 'O'], ['on', 'O'], ['Good', 'I-MISC'], ['Day', 'I-MISC'], [',', 'O'], ['and', 'O'], ['Wardanceis', 'I-MISC'], ['cover', 'O'], ['of', 'O'], ['a', 'O'], ['song', 'O'], ['by', 'O'], ['UK', 'I-LOC'], ['post', 'O'], ['punk', 'O'], ['in

/home/pritish/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
lengths = [len(x) for x in all_x]
print ('Input sequence length range: ', max(lengths), min(lengths))

short_x = [x for x in all_x if len(x) < 64]

X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]
print(X[0], "\n", y[0])
all_text = [c for x in X for c in x]

print(X)

Input sequence length range:  144 1
['010', 'is', 'the', 'tenth', 'album', 'from', 'Japanese', 'Punk', 'Techno', 'band', 'The', 'Mad', 'Capsule', 'Markets', '.'] 
 ['I-MISC', 'O', 'O', 'O', 'O', 'O', 'I-MISC', 'O', 'O', 'O', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O']
[['010', 'is', 'the', 'tenth', 'album', 'from', 'Japanese', 'Punk', 'Techno', 'band', 'The', 'Mad', 'Capsule', 'Markets', '.'], ['This', 'album', 'proved', 'to', 'be', 'more', 'commercial', 'and', 'more', 'techno-based', 'than', 'Osc-Dis', ',', 'with', 'heavily', 'synthesized', 'songs', 'like', 'Introduction', '010', 'and', 'Come', '.'], ['Founding', 'member', 'Kojima', 'Minoru', 'played', 'guitar', 'on', 'Good', 'Day', ',', 'and', 'Wardanceis', 'cover', 'of', 'a', 'song', 'by', 'UK', 'post', 'punk', 'industrial', 'band', 'Killing', 'Joke', '.'], ['XXX', 'can', 'of', 'This', 'had', 'a', 'different', 'meaning', ',', 'and', 'most', 'people', 'did', "n't", 'understand', 'what', 'the', 'song', 'was', 'about', '.'], ['it', 'was',

In [3]:
print(all_text)

['010', 'is', 'the', 'tenth', 'album', 'from', 'Japanese', 'Punk', 'Techno', 'band', 'The', 'Mad', 'Capsule', 'Markets', '.', 'This', 'album', 'proved', 'to', 'be', 'more', 'commercial', 'and', 'more', 'techno-based', 'than', 'Osc-Dis', ',', 'with', 'heavily', 'synthesized', 'songs', 'like', 'Introduction', '010', 'and', 'Come', '.', 'Founding', 'member', 'Kojima', 'Minoru', 'played', 'guitar', 'on', 'Good', 'Day', ',', 'and', 'Wardanceis', 'cover', 'of', 'a', 'song', 'by', 'UK', 'post', 'punk', 'industrial', 'band', 'Killing', 'Joke', '.', 'XXX', 'can', 'of', 'This', 'had', 'a', 'different', 'meaning', ',', 'and', 'most', 'people', 'did', "n't", 'understand', 'what', 'the', 'song', 'was', 'about', '.', 'it', 'was', 'later', 'explained', 'that', 'the', 'song', 'was', 'about', 'Cannabis', '(', "'", 'can', 'of', 'this', "'", 'sounding', 'like', 'Cannabis', 'when', 'said', 'faster', ')', 'it', 'is', 'uncertain', 'if', 'they', 'were', 'told', 'to', 'change', 'the', 'lyric', 'like', 'they',

In [11]:
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}
print ('Vocabulary size:', len(word2ind), len(label2ind))

maxlen = max([len(x) for x in X])
print ('Maximum sequence length:', maxlen)
print(label2ind)

def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result

X_enc = [[word2ind[c] for c in x] for x in X]
max_label = max(label2ind.values()) + 1
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]

X_enc = pad_sequences(X_enc, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

Vocabulary size: 8285 5
Maximum sequence length: 63
{'I-LOC': 3, 'I-MISC': 2, 'I-ORG': 1, 'I-PER': 5, 'O': 4}


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=11*32, train_size=45*32, random_state=42)
print ('Training and testing tensor shapes:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)

max_features = len(word2ind)
embedding_size = 128
hidden_size = 32
out_size = len(label2ind) + 1

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model.add(LSTM(hidden_size, return_sequences=True))  
model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()
batch_size = 32
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Raw test score:', score)

Training and testing tensor shapes: (1440, 63) (352, 63) (1440, 63, 6) (352, 63, 6)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 63, 128)       1060480     embedding_input_2[0][0]          
____________________________________________________________________________________________________


/usr/local/lib/python3.5/dist-packages/keras/layers/core.py:1112: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


lstm_2 (LSTM)                    (None, 63, 32)        20608       embedding_2[0][0]                
____________________________________________________________________________________________________
timedistributeddense_2 (TimeDistr(None, 63, 6)         198         lstm_2[0][0]                     
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 63, 6)         0           timedistributeddense_2[0][0]     
Total params: 1081286
____________________________________________________________________________________________________
Train on 1440 samples, validate on 352 samples
Epoch 1/10
1440/1440 [==============================] - 1s - loss: 1.2137 - val_loss: 0.7896
Epoch 2/10
1440/1440 [==============================] - 1s - loss: 0.7062 - val_loss: 0.6196
Epoch 3/10
1440/1440 [==============================] - 1s - loss: 0.5304 - val_loss: 0.4877
Epoch 4/10
1440/1440 [========================

In [25]:
def score(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

pr = model.predict_classes(X_train)
yh = y_train.argmax(2)
fyh, fpr = score(yh, pr)
print ('Training accuracy:', accuracy_score(fyh, fpr))
print ('Training confusion matrix:')
print (confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

pr = model.predict_classes(X_test)
yh = y_test.argmax(2)
fyh, fpr = score(yh, pr)
print ('Testing accuracy:', accuracy_score(fyh, fpr))
print ('Testing confusion matrix:')
print (confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

1440/1440 [==============================] - 0s     
Training accuracy: 0.982971405449
Training confusion matrix:
[[ 1427     7    32    73     3]
 [   26   769    74   135    16]
 [   40     9  1043    19     5]
 [   21    19     7 24703     2]
 [    1     0     0    16  1209]]
352/352 [==============================] - 0s     
Testing accuracy: 0.906838185511
Testing confusion matrix:


(array([ 0.60107817,  0.56953642,  0.7679558 ,  0.93946881,  0.80097087]),
 array([ 0.62464986,  0.31501832,  0.51865672,  0.98065764,  0.58303887]),
 array([ 0.61263736,  0.40566038,  0.61915367,  0.95962145,  0.67484663]),
 array([ 357,  273,  268, 6204,  283]))

In [22]:
ip = [word2ind['John'], word2ind['was'], word2ind['a'], word2ind['member'], word2ind['of'], word2ind['US'], word2ind['Army']]
i=maxlen-len(ip)
temp=[0]*i
ip=temp+ip
print(ip)
op = K.function([model.layers[0].input], model.layers[3].output)
out = op([[ip]])
print(out.shape)
print(out)
while i<maxlen:
    print(out[0][i].tolist().index(max(out[0][i])))
    i=i+1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5540, 1388, 2076, 6458, 6468, 3347, 1876]
(1, 63, 6)
[[[  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796e-01   1.68050587e-01   1.65499404e-01   1.62778705e-01
     1.78782552e-01   1.70634955e-01]
  [  1.54253796

In [23]:
print(label2ind)

{'I-LOC': 3, 'I-MISC': 2, 'I-ORG': 1, 'I-PER': 5, 'O': 4}


In [24]:
print(len(word2ind))
print(word2ind)

8285
{'Defensive': 4198, 'Because': 0, '-DOCSTART-': 4199, 'acoustic': 1, 'employs': 4201, 'An': 3, 'Athens': 63, 'Thanksgiving': 4203, 'hit': 4, 'interface': 4204, 'releasing': 8218, 'output': 6, 'Vertinskaya': 2, 'szlachta': 9, 'train': 4205, 'championship': 10, 'charting': 12, 'handed': 4207, 'AMEL': 13, 'heard': 4208, 'goals': 8163, 'handcrafts': 4209, 'Hunter': 4210, 'Pedr': 14, 'businesses': 4211, 'Safeway': 4212, 'Radio': 7612, 'injuring': 15, 'renamed': 16, 'novel': 17, "'s": 4214, 'Arizona': 18, 'Martindale': 19, 'history': 20, 'sophomore': 21, 'occasions': 22, 'organized': 23, 'doubled': 6912, 'Bresh': 24, 'Technology': 28, 'Water': 26, 'tiebreaker': 4218, 'Solo': 29, 'then-infant': 4220, 'gigged': 30, 'muscular': 7616, 'Trust': 6098, 'Chase': 5741, 'first-round': 4222, 'Portchester': 4223, 'injects': 4224, 'expected': 4876, 'BlackBerry': 31, 'Cochabamba': 4225, 'Cascade': 32, 'Captain': 33, 'once': 34, 'led': 4227, 'surfers': 4228, 'insists': 35, 'Whale': 36, 'charge': 7703,